In [ ]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt, seaborn as sns
from pathlib import Path

# Ajusta:
DATA_PATH = Path("/Users/alonso/Downloads/Tesis_SIEM_ML/data/raw/logs_unificados_limpios.csv")  
TARGET = "action"
TIME_COL = "timestamp"                           

df = pd.read_csv(DATA_PATH)
df.head()

In [ ]:
df.info()
df.describe(include="all").T
missing = df.isna().mean().sort_values(ascending=False)
missing.head(20)
dup_rate = df.duplicated().mean()
dup_rate

In [ ]:
num_cols = df.select_dtypes(include=np.number).columns.tolist()
cat_cols = [c for c in df.columns if c not in num_cols + [TARGET]]

# numéricas
for c in num_cols:
    if c == TARGET: 
        continue
    df[c].plot(kind="hist", bins=30, title=f"Distribución {c}")
    plt.xlabel(c); plt.show()

# categóricas
for c in cat_cols:
    vc = df[c].value_counts().head(20)
    vc.plot(kind="bar", title=f"Top categorías {c}")
    plt.xlabel(c); plt.show()

In [ ]:
if df[TARGET].dropna().isin([0,1]).all():
    corr = df[num_cols+[TARGET]].corr(numeric_only=True)[TARGET].sort_values(ascending=False)
    corr

if TARGET in df.columns:
    cls_vc = df[TARGET].value_counts(dropna=False)
    cls_pct = (cls_vc / len(df)).round(3)
    display(cls_vc, cls_pct)    

time_col = "timestamp"   # ajusta
if time_col in df.columns:
    df[time_col] = pd.to_datetime(df[time_col], errors="coerce")
    df = df.sort_values(time_col)
    # proporción de clase por mes
    cls_by_month = df.groupby(pd.Grouper(key=time_col, freq="M"))[TARGET].mean()
    cls_by_month.plot(title="Proporción de clase positiva por mes"); plt.show()


In [ ]:
drop_cols = []  
df1 = df.drop(columns=[c for c in drop_cols if c in df.columns]).copy()

df1 = df1.drop_duplicates()

df1 = df1.dropna(subset=[TARGET])  

df1.to_csv("data/processed/dataset_clean.csv", index=False)
df1.shape
